In [24]:
from analytics_functions import *
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from collections import defaultdict

df = pd.read_csv('UM_plays.txt',sep='\t')

# Measuring team defensive efficiency using foul rates

Definition: **Time since last rest** is a measure of how long a player has been on court since their last break. Both benchings and halftime qualify as 'rests', timeouts do not. We are looking for extended rests, therefore timeouts don't qualify. It may not be a perfect measure, (subbing player out for defensive possesion to avoid a foul, then subbing in on offense is not much of a break) but these events are rare and short, so they do not affect results much.

Here we are looking to measure defensive efficiency using foul rates (fouls per minute), as our team grows more tired. Foul rates should act as a good proxy of defensive efficiency, as players that get beat off the dribble, are slow to move their feet, reach, get in improper rebounding position,..., are more likely to foul. Measuring this foul rate as a team should act as a good measure, as a tired, lazy defender getting beat off the dribble might cause a teammate under the basket to pick up a foul contesting the drive. We group time into different buckets to avoid small sample sizes. For example, we look at the foul rate when the average time since last rest for the players on court falls into the range (5 minutes - 6 minutes). We are using 1 minute windows. Foul rates are good measure of efficiency here because they normalize for how common it is to be in a given time window. An example is given below.

Suppose we want to compare how often we foul when our average time

The calculation steps are as follows.
1. Calculate the average time since last rest for every play in our play-by-play data.
2. Filter our data to show only sequences where the average time since last rest falls into our window (0 mins - 1 min, 1 min - 2 min, etc.).
3. Calculate the number of fouls and total number of minutes played in that time window.
4. Report fouls per minute as (number of fouls)/(minutes played)


Time windows with less than 10 minutes played are dropped. Ex. We have only seen 8 total minutes played where the average time since last rest of players on court is in range (13 mins - 14 mins), this is not included.

In [25]:
foul_list, durations, windows = team_foul_rate(df)
durations = [x/sum(durations)*(31*40) for x in durations] #Scale durations to 31games*40mins. Fixes small errors
durations = [np.nan if x==0 else x for x in durations]
foul_rates=[foul/d for foul,d in zip(foul_list,durations)]

season_foul_rate = sum(foul_list)/1240

duration_cutoff=10
rate_df=pd.DataFrame()
rate_df['foul_rates']=foul_rates
rate_df['windows']= windows
rate_df['durations']=durations

rate_df=rate_df[rate_df.durations >=duration_cutoff]
fig = go.Figure(data=go.Scatter(x=[x[0] for x in list(rate_df.windows)], y=rate_df.foul_rates, mode='markers'))

fig.add_shape(type="line",
    x0=-0.5, y0=season_foul_rate, x1=7.5, y1=season_foul_rate,
    line=dict(
        color="red",
        width=4,
        dash="dot",
    ))
              
fig.update_layout(title='Team Foul Rate as a function of Time Since Last Rest',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [x[0] for x in list(rate_df.windows)],
        ticktext = [str(x) for x in list(rate_df.windows)]
    )
)

fig.add_annotation(x=6, y=0.5,
            text="Season Average",
            showarrow=False)

fig.update_xaxes(title='Average Time Since Last Rest (Team) in Minutes 2019-20')
fig.update_yaxes(title='Fouls per Min (Team) ')
fig.show()


In [26]:
#Player-centric version

all_players=['PRIDGETT,SAYEED', 'MANUEL,KENDAL', 'VAZQUEZ,JOSH',
       'ANDERSON,MACK', 'OWENS,KYLE', 'CARTER-HOLLI,DERRICK', 
       'FALLS,TIMMY', 'EGUN,EDDY', 'SAMUELSON,JARED', 'SELCUK,YAGIZHAN',
       'JONES,PETER']

p_foul_dict=defaultdict(defaultdict(int))

for player in all_players:
    p_season_rate = player_season_foul_rate(df,player)
    fouls_p,durations_p,windows_p =player_foul_rate(df, player)
    print(fouls_p)

TypeError: first argument must be callable or None